In [1]:
# %store -r final_dataframe

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from pandas import DataFrame
from math import sqrt



In [3]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [11]:
os_path = '/home/dsp_team_gregory/gregory-p1/'
df = pd.read_csv (os_path+'data/X_train.txt.csv')
df.drop(df.columns[[0]], axis=1, inplace=True)
df

,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,11.502672,0.916191,0.629950,0.854301,0.621109,0.572481,1.491768,0.928348,0.544852,0.847670,...,0.915086,2.675631,1.052128,1.286425,1.182538,5.350157,1.668816,7.575981,5.450728,48.779153
1,21.232641,2.284399,1.438939,4.838461,0.792411,0.680788,2.049645,1.907534,1.346104,1.367103,...,1.016762,6.352552,1.453306,1.815804,2.071100,8.313134,6.175723,2.986186,5.848591,75.041450
2,7.886536,3.567508,3.566403,3.558667,3.487935,3.575244,8.021631,3.617241,3.542089,3.521091,...,3.468042,3.568613,3.575244,3.592927,3.611715,3.817278,3.623872,3.514460,6.061890,12.624425
3,7.982686,3.794069,3.464727,3.579665,3.528827,3.408363,7.884005,3.513355,3.505618,3.590717,...,3.708971,3.602874,3.675815,3.634924,3.505618,3.780807,3.692393,3.574139,6.203353,12.676369
4,5.039603,0.670842,0.543747,0.769202,0.536010,0.551483,1.221431,0.792411,0.681894,0.658685,...,0.600111,1.367103,0.696261,0.773623,0.904034,1.883220,1.110702,0.926137,4.567692,30.875302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8142,2.575060,2.117517,2.099834,2.100940,2.011420,2.041260,4.603099,2.204826,2.213668,2.165040,...,2.131885,2.162829,2.262295,2.130779,2.138516,2.125253,2.183828,2.046786,2.347394,7.595875
8143,6.454228,2.075521,0.974765,1.284215,1.024498,1.018972,1.698207,1.787070,1.347210,2.062258,...,1.959477,2.657948,2.018051,3.550930,2.355130,4.019525,2.127464,2.333027,3.126543,22.254929
8144,2.104255,1.682078,1.683183,1.657764,1.689814,1.589243,3.602853,1.584822,1.661080,1.612452,...,1.704181,1.589243,1.651133,1.749494,1.974950,1.715233,1.724075,1.631240,1.872168,4.065943
8145,5.184381,3.291214,3.136489,4.253822,2.997237,3.002763,6.905877,3.579665,3.884694,3.508934,...,3.182907,4.199669,3.965371,4.314607,3.439308,3.280162,3.486830,3.926690,4.007368,682.491492


In [12]:
def addTargetLabelToDataset(df):
    df['target'] = target_dataframe['target']
    return df

In [13]:
target_dataframe = pd.read_csv('https://storage.googleapis.com/uga-dsp/project1/files/y_train.txt', sep=" ", header=None)
target_dataframe.columns = ['target']
# target_dataframe

In [19]:

df = addTargetLabelToDataset(df)


In [48]:
statistics_of_columns = []
for column in df:
        new_d = [df[column].mean(),df[column].std(),len(df[column])]
        statistics_of_columns.append(new_d)
del statistics_of_columns[-1]

[[17.313334785995284, 25.19281840062475, 8147],
 [3.347988729721373, 5.163951723331314, 8147],
 [2.6406253549939755, 9.006532967240648, 8147],
 [3.596586994743586, 4.071689848873464, 8147],
 [1.9816245200098188, 6.52714300195113, 8147],
 [2.4249685787141235, 9.773420216057403, 8147],
 [5.024742484857924, 19.578403325590727, 8147],
 [3.4328709130310564, 4.392295316539734, 8147],
 [3.2632346041193023, 10.159467604385469, 8147],
 [2.1320495953455203, 7.142850060156519, 8147],
 [2.116606432685402, 8.587161906293005, 8147],
 [2.156798103887078, 7.897839259031429, 8147],
 [2.059270628912724, 7.428285258168057, 8147],
 [2.7547380666119983, 10.008860475032158, 8147],
 [2.629268237049224, 8.559066219376016, 8147],
 [4.110489358149989, 4.197259631594438, 8147],
 [2.0668235918841265, 7.037005117487586, 8147],
 [2.692082715513097, 10.105234079843033, 8147],
 [2.0651516520044138, 8.246584249894022, 8147],
 [3.2796485093172874, 4.731160610092305, 8147],
 [3.7400077728724574, 4.505780610986432, 8147]